<a href="https://colab.research.google.com/github/davidjeans1/Davids_Portfolio/blob/main/data_processing_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Pipeline Process

1. Get ALL Stocks and Glassnode data since 2008

2. Get prices data for specific coin

3. Create TA dataframe based upon pricing data

4. Left Join prices, TA, stocks, glassnode

5. Perform dimension reduction (maybe move after step 3)

#Getting Nomics Pricing Data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import date, timedelta, datetime
import requests
import io

def requestData(coin, isTrainingSet=True, delta=150):
    key = "6d0dc559240e6cb9142b14d63b0591cb"
    if isTrainingSet:
        endDate = (date.today() - timedelta(days=delta)).isoformat()
        url = "https://api.nomics.com/v1/candles?key=" + key + "&interval=1d&currency=" + coin + "&end=" + endDate + "T00%3A00%3A00Z"
    else:
        startDate = (date.today() - timedelta(days=delta+60)).isoformat() #60 day overlap between train and test set
        endDate = (date.today() - timedelta(days=1)).isoformat()
        url = "https://api.nomics.com/v1/candles?key=" + key + "&interval=1d&currency=" + coin + "&start=" + startDate + "T00%3A00%3A00Z&end=" + endDate + "T00%3A00%3A00Z"
        print(url)
    rawData = pd.read_json(url)
    if (rawData.empty):
        raise Exception(coin + " does not exist")
    return rawData

#Getting AlphaVantage Stocks Data

In [3]:

ALPHA_API = "A92RWPBGHVN1IM2M"

def request_stocks(ticker):
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&outputsize=full&apikey={ALPHA_API}'
  r = requests.get(url).json()
#  if r['Meta Data']:
 #   del r['Meta Data']
  data = pd.DataFrame.from_dict(r['Time Series (Daily)']).transpose()
  data.columns = ['open', 'high', 'low', 'close', 'adjusted close', 'volume', 'divided amount', 'split coefficient']
  data.index = pd.to_datetime(data.index)
  return pd.DataFrame(data)

UUP = request_stocks('UUP')['adjusted close']
UUP = UUP.rename('UUP').astype(float)
SPY = request_stocks("SPY")['adjusted close']
SPY = SPY.rename('SPY').astype(float)


#Get Glassnode Data and combine


In [39]:
#for function to get all variables
GLASSNODE_API = "1yGjKl8gmits6q0X3gTK7pu7SKM"

def all_glassnode_data():
  #no timestamps in count
  res = requests.get('https://api.glassnode.com/v2/metrics/endpoints',
      params={"a": "BTC",'api_key': GLASSNODE_API, "s": 1231027200, "i": "24h", "f": "CSV", "timestamp_format": "humanized"})
  glassnode = pd.read_json(io.StringIO(res.text))
  metrics = glassnode['path']
  
  fund_df = pd.DataFrame()
  for metric in metrics:
    try:
      res = requests.get(f'https://api.glassnode.com{metric}',
      params={"a": "BTC",'api_key': GLASSNODE_API, "s": 1231027200, "i": "24h", "f": "CSV", "timestamp_format": "humanized"})
      glassnode = pd.read_csv(io.StringIO(res.text))
      glassnode.index = [datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").date() for date in glassnode['timestamp']]
      glassnode = glassnode.rename(columns = {"value": metric})
      glassnode = glassnode.drop(columns = ['timestamp'], axis = 1)
      fund_df = fund_df.join(glassnode, how = 'outer')
      print(metric)
    except Exception:
      print(f'{metric} did not work')
      pass
  fund_df = fund_df.join(UUP).join(SPY))
  return fund_df



In [40]:
test = all_glassnode_data()

/v1/metrics/addresses/sending_to_exchanges_count
/v1/metrics/addresses/receiving_from_exchanges_count
/v1/metrics/addresses/count
/v1/metrics/addresses/sending_count
/v1/metrics/addresses/receiving_count
/v1/metrics/addresses/active_count
/v1/metrics/addresses/new_non_zero_count
/v1/metrics/addresses/accumulation_count
/v1/metrics/addresses/accumulation_balance
/v1/metrics/addresses/non_zero_count
/v1/metrics/addresses/min_point_zero_1_count
/v1/metrics/addresses/min_point_1_count
/v1/metrics/addresses/min_1_count
/v1/metrics/addresses/min_10_count
/v1/metrics/addresses/min_100_count
/v1/metrics/addresses/min_1k_count
/v1/metrics/addresses/min_10k_count
/v1/metrics/addresses/min_32_count did not work
/v1/metrics/addresses/supply_balance_less_0001
/v1/metrics/addresses/supply_balance_0001_001
/v1/metrics/addresses/supply_balance_001_01
/v1/metrics/addresses/supply_balance_01_1
/v1/metrics/addresses/supply_balance_1_10
/v1/metrics/addresses/supply_balance_10_100
/v1/metrics/addresses/sup

In [65]:
pd.set_option('display.max_rows', 500)

fund_df = test.join(UUP).join(SPY)
fund_df.to_csv("test.csv")

#Making prices and TA dataset

In [9]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3
import talib

#request coin prices first
price_df = requestData("BTC")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3737    0  3737    0     0  17880      0 --:--:-- --:--:-- --:--:-- 17880
100  503k  100  503k    0     0   912k      0 --:--:-- --:--:-- --:--:--  912k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3721    0  3721    0     0  19481      0 --:--:-- --:--:-- --:--:-- 19380
100  406k  100  406k    0     0   854k      0 --:--:-- --:--:-- --:--:--  854k


In [23]:

def TA_library(price_df):
  TA_df = pd.DataFrame()
  TA_array = []
  df1 = pd.DataFrame()
  open = price_df['open']
  high = price_df['high']
  low = price_df['low']
  close = price_df['close']
  volume = price_df['volume']

  TA_array = np.array([
      #overlap studies
      talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)[0],
      talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)[1],
      talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)[2],
      talib.SMA(close),
      talib.DEMA(close), 
      talib.T3(close, timeperiod=5, vfactor=0),
      talib.TEMA(close, timeperiod=30),
      talib.TRIMA(close, timeperiod=30),
      talib.EMA(close),
      talib.HT_TRENDLINE(close),
      talib.KAMA(close),
      talib.MAMA(close)[0],
      talib.MAMA(close)[1],
      talib.MIDPOINT(close),
      talib.SAR(high, low, acceleration=0, maximum=0),
      talib.WMA(close),

      #momentum indicators
      talib.ADX(high, low, close),
      talib.APO(close, fastperiod=12, slowperiod=26, matype=0),
      talib.AROON(high, low, timeperiod=14)[0],
      talib.AROON(high, low, timeperiod=14)[1],
      talib.AROONOSC(high, low, timeperiod=14),
      talib.BOP(open, high, low, close),
      talib.CMO(close, timeperiod=14),
      talib.DX(high, low, close, timeperiod=14),
      talib.CCI(high, low, close, timeperiod=14),
      talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)[0],
      talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)[1],
      talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)[2],
      talib.MFI(high, low, close, volume, timeperiod=14),
      talib.MOM(close, timeperiod=10),
      talib.RSI(close, timeperiod=14),
      talib.PPO(close, fastperiod=12, slowperiod=26, matype=0),
      talib.ROC(close, timeperiod=10),
      talib.ROCR(close, timeperiod=10),
      talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[0],
      talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[1],

      talib.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)[0],
      talib.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)[1],

      talib.STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)[0],
      talib.STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)[1],
      talib.WILLR(high, low, close, timeperiod=14),
      talib.TRIX(close, timeperiod=30),
      talib.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28),
      #Volume Indicators
      talib.AD(high, low, close, volume),
      talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10),
      talib.OBV(close, volume),

      #Volatility Indicators
      talib.ATR(high, low, close, timeperiod=14),
      talib.NATR(high, low, close, timeperiod=14),
      talib.TRANGE(high, low, close),

      # #Price Transform
      talib.AVGPRICE(open, high, low, close),
      talib.MEDPRICE(high, low),
      talib.TYPPRICE(high, low, close),
      talib.WCLPRICE(high, low, close),

      #Cycle Indicators
      talib.HT_DCPERIOD(close),
      talib.HT_DCPHASE(close),
      talib.HT_PHASOR(close)[0],
      talib.HT_PHASOR(close)[1],
      talib.HT_SINE(close)[0],
      talib.HT_TRENDMODE(close),

      #Pattern Recognition
      talib.CDL2CROWS(open, high, low, close),
      talib.CDL3BLACKCROWS(open, high, low, close),
      talib.CDL3INSIDE(open, high, low, close),
      talib.CDL3LINESTRIKE(open, high, low, close),
      talib.CDL3OUTSIDE(open, high, low, close),
      talib.CDL3STARSINSOUTH(open, high, low, close),
      talib.CDL3WHITESOLDIERS(open, high, low, close),
      talib.CDLABANDONEDBABY(open, high, low, close, penetration=0),
      talib.CDLADVANCEBLOCK(open, high, low, close),
      talib.CDLBELTHOLD(open, high, low, close),
      talib.CDLBREAKAWAY(open, high, low, close),
      talib.CDLCLOSINGMARUBOZU(open, high, low, close),
      talib.CDLCONCEALBABYSWALL(open, high, low, close),
      talib.CDLCOUNTERATTACK(open, high, low, close),
      talib.CDLDOJI(open, high, low, close),
      talib.CDLDOJISTAR(open, high, low, close),
      talib.CDLDRAGONFLYDOJI(open, high, low, close),
      talib.CDLENGULFING(open, high, low, close),
      talib.CDLEVENINGDOJISTAR(open, high, low, close, penetration=0),
      talib.CDLEVENINGSTAR(open, high, low, close, penetration=0), 
      talib.CDLGAPSIDESIDEWHITE(open, high, low, close),
      talib.CDLGRAVESTONEDOJI(open, high, low, close), 
      talib.CDLHAMMER(open, high, low, close),
      talib.CDLHANGINGMAN(open, high, low, close),
      talib.CDLHARAMI(open, high, low, close),
      talib.CDLHARAMICROSS(open, high, low, close),
      talib.CDLHIGHWAVE(open, high, low, close),
      talib.CDLHIKKAKE(open, high, low, close),
      talib.CDLHOMINGPIGEON(open, high, low, close),
      talib.CDLIDENTICAL3CROWS(open, high, low, close),
      talib.CDLINNECK(open, high, low, close),
      talib.CDLINVERTEDHAMMER(open, high, low, close), 
      talib.CDLKICKING(open, high, low, close),
      talib.CDLKICKINGBYLENGTH(open, high, low, close),
      talib.CDLLADDERBOTTOM(open, high, low, close),
      talib.CDLLONGLEGGEDDOJI(open, high, low, close),
      talib.CDLLONGLINE(open, high, low, close),
      talib.CDLMARUBOZU(open, high, low, close),
      talib.CDLMATCHINGLOW(open, high, low, close),
      talib.CDLMATHOLD(open, high, low, close, penetration=0),
      talib.CDLMORNINGDOJISTAR(open, high, low, close, penetration=0),
      talib.CDLMORNINGSTAR(open, high, low, close, penetration=0),
      talib.CDLONNECK(open, high, low, close),
      talib.CDLPIERCING(open, high, low, close),
      talib.CDLRICKSHAWMAN(open, high, low, close),
      talib.CDLRISEFALL3METHODS(open, high, low, close),
      talib.CDLSEPARATINGLINES(open, high, low, close),
      talib.CDLSHOOTINGSTAR(open, high, low, close),
      talib.CDLSHORTLINE(open, high, low, close),
      talib.CDLSPINNINGTOP(open, high, low, close),
      talib.CDLSTALLEDPATTERN(open, high, low, close),
      talib.CDLSTICKSANDWICH(open, high, low, close),
      talib.CDLTAKURI(open, high, low, close),
      talib.CDLTASUKIGAP(open, high, low, close),
      talib.CDLTHRUSTING(open, high, low, close),
      talib.CDLTRISTAR(open, high, low, close),
      talib.CDLUNIQUE3RIVER(open, high, low, close),
      talib.CDLUPSIDEGAP2CROWS(open, high, low, close),
      talib.CDLXSIDEGAP3METHODS(open, high, low, close),

      #Statistic Function
      talib.BETA(high, low, timeperiod=5),
      talib.CORREL(high, low, timeperiod=30),
      talib.LINEARREG(close, timeperiod=14),
      talib.LINEARREG_ANGLE(close, timeperiod=14),
      talib.LINEARREG_INTERCEPT(close, timeperiod=14),
      talib.LINEARREG_SLOPE(close, timeperiod=14),
      talib.STDDEV(close, timeperiod=5, nbdev=1),
      talib.TSF(close, timeperiod=14),
      talib.VAR(close, timeperiod=5, nbdev=1),
  ], dtype=object)


  #max technical indicator length should be 30
  TA_df = pd.DataFrame(TA_array).T

  #add engineered features
  plr = pd.DataFrame([talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)[0],
                    talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)[1],
                    talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)[2],
                    talib.CCI(high, low, close, timeperiod=24),
                    talib.MOM(close, timeperiod=10),
                    talib.ROC(close, timeperiod=10),
                    talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10),  
                    ]).T

  plr[plr > 0] = 1
  plr[plr < 0] = -1

  minmax = [
            volume,
            talib.SMA(close, timeperiod=10),
            talib.RSI(close, timeperiod=5),
            talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[0],
            talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[1],
            talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10),
                         
  ]

  scaler = MinMaxScaler()
  scaled_df = pd.DataFrame(scaler.fit_transform(minmax)).T

  # flux = [
  #           talib.SMA(close, timeperiod=10),
  #           talib.MOM(close, timeperiod=10),
  #           talib.ROC(close, timeperiod=10),
  #           talib.RSI(close, timeperiod=5),
  #           talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[0],
  #           talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)[1],
  # ]

  

  TA_df = pd.concat([TA_df, plr, scaled_df], axis=1)
  
  return TA_df

  #then perform rfe, make sure to rfe.fit_transform(model, 4), run on random 60% sample and pick all high weighted features (SVR model), resample for each model
  #take top i features, and standardize data before inputting to PCA using minmax scaling
  #dont need to preprocess again before lstm model fitting

  #in china paper, they do 54 -> RFE -> 30 -> PCA -> 20

  #feature amount differs drastically based on term lengths based on RFE

  #look at Mcnally paper



  #look into querying thegraph data

#Format input data

In [104]:
def formatData(prices_df, TA_df, glassnode_df):
    data = pd.concat([prices_df, TA_df], axis = 1)[:len(prices_df)]
    data['timestamp'] = data['timestamp'].dt.date
    data.set_index('timestamp', inplace = True)
    data_new = data.join(glassnode_df).reset_index()
    data_new = data_new.drop(['timestamp','transparent_open', 'transparent_high', 'transparent_low', 'transparent_close', 'transparent_volume', 'volume_transparency'], axis = 1).T.reset_index(drop=True).T

    return data_new

#Fill Missing Values

In [84]:
# fills in blank cells using mean of k=2 nearest neighbors
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer

# input dataFrame and output dataFramee
def fillMissingValues(data):
#     strAttribs = ["Date"]
    numAttribs = data.columns #saves column titles

    # pipeline different transformations
    pipeline = ColumnTransformer([
#             ("date", 'passthrough', strAttribs),
            ("num", KNNImputer(n_neighbors=2), numAttribs)
        ])

    dataArray = pipeline.fit_transform(data)

    # convert back to dataFrame with original headers and index
    dataComplete = pd.DataFrame(dataArray, columns=data.columns, index=data.index)
    print(dataComplete)
    return dataComplete

#MinMax Scaling

In [85]:
# MinMaxScaler to normalize data
def minMax(dataArray):
    scaler = MinMaxScaler()
    dataScaled = scaler.fit_transform(dataArray)
    return dataScaled, scaler

# used to match predictions to same scale as training data
def minMaxMatch(dataArray, scaler):
    dataScaled = scaler.transform(dataArray)
    return dataScaled

# unscale yTest, i.e using first column of scaler
def minMaxUnscale(dataArray, scaler):
    dataUnscaled = scaler.inverse_transform(dataArray)
    return dataUnscaled

#Build Input Data

In [114]:
def buildInputData(trainingData, days=1, version=1):
    '''Build input data using 60 days worth of prior data for each prediction'''
    trainingData = np.asarray(trainingData)
    xTrain = []
    yTrain = []

    # version 1 uses consecutive one day open data to make future predictions
    if (version == 1):
        if (days == 1):
            for i in range(60, trainingData.shape[0]):
                xTrain.append(trainingData[i-60:i])
                yTrain.append(trainingData[i,0])
        elif (days == 7):
            for i in range(67, trainingData.shape[0]):
                xTrain.append(trainingData[i-67:i-7])
                yTrain.append(trainingData[i,0])
        elif (days == 30):
            for i in range(90, trainingData.shape[0]):
                xTrain.append(trainingData[i-90:i-30])
                yTrain.append(trainingData[i,0])
        else:
            raise Exception("only 1, 7, or 30 days allowed")
       
    # version 2 uses data from 1 day each week to make a weekly prediction
    elif (version == 2):
        if (days == 7):
            skip = len(trainingData) % 7
            weeks = len(trainingData) // 7

            for i in range (8, weeks): # using 8 weeks to predict
                xData = []
                for j in range (8, 0, -1):
                    xData.append(trainingData[(i-j)*7 + skip])
                xTrain.append(xData)
                yTrain.append(trainingData[i*7 + skip, 0])

        else: 
            raise Exception("not a valid day")
    else:
        raise Exception("not a valid version")
        
    xTrain, yTrain = np.array(xTrain), np.array(yTrain)
    return xTrain, yTrain

#Combine Data and format for RFE

In [119]:
data = pd.DataFrame(requestData('BTC'))

TA = TA_library(requestData('BTC'))


full_df = formatData(data, TA, fund_df)
full_df = fillMissingValues(full_df)
#xdata, ydata = buildInputData(full_df)

               0             1    ...        648         649
0        10.900000     10.900000  ...  20.425916   93.738242
1        11.850000     11.850000  ...  20.377467   92.207454
2        11.700000     11.700000  ...  20.425916   94.462705
3        11.700000     11.700000  ...  20.421071   93.009685
4        11.700000     11.700000  ...  20.416226   92.281128
...            ...           ...  ...        ...         ...
3525  59049.718571  59049.718571  ...  24.425000  414.062855
3526  58388.407802  58575.084875  ...  24.270000  415.215293
3527  56008.766267  56947.009815  ...  24.250000  411.509610
3528  56788.657773  56788.657773  ...  24.430000  402.766981
3529  49735.942040  50957.518678  ...  24.390000  407.605231

[3530 rows x 650 columns]


#Selecting Data (RFE and PCA)

In [120]:
#making RFE
from sklearn.svm import SVR
from sklearn.feature_selection import RFECV
estimator=SVR(kernel='linear')
selector = RFECV(estimator, step = 1, cv = 5)
selector = selector.fit(xdata, ydata)


ValueError: ignored